In [ ]:
import pandas as pd
import torch
from torch import cuda
import logging
import random
import string
import regex as re
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from nltk import sent_tokenize, word_tokenize

In [ ]:
import numpy as np
df = pd.read_csv('balanced_dataset.csv')
np.random.seed(42)
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_eval = df[~msk]

In [ ]:
all_sents = []
for i in df.sents_l:
    all_sents.append(i)
for j in df.sents_r:
    all_sents.append(j)

In [ ]:
for i in range(len(all_sents)):
    all_sents[i] =  all_sents[i].replace('\n', ' ')

In [ ]:
!git clone https://github.com/akutuzov/webvectors.git
!git clone https://github.com/dialogue-evaluation/RuArg.git

!pip install wget
!pip install ufal.udpipe

Cloning into 'webvectors'...
remote: Enumerating objects: 2022, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 2022 (delta 110), reused 113 (delta 54), pack-reused 1837
Receiving objects: 100% (2022/2022), 4.78 MiB | 17.98 MiB/s, done.
Resolving deltas: 100% (1348/1348), done.
Cloning into 'RuArg'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 50 (delta 10), reused 12 (delta 2), pack-reused 0
Receiving objects: 100% (50/50), 629.00 KiB | 2.55 MiB/s, done.
Resolving deltas: 100% (10/10), done.
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=7f927779fa40d2fbe1bc4892a9a3f6689d83d43be55c33f511930e7eaf95ca65
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━

In [ ]:
#создаем архив с текстами (предложениями)
import gzip

with open('corpus_sent.txt', 'w') as fp:
    for s in all_sents:
        fp.write("%s\n" % s)

f_in = open('corpus_sent.txt')
f_out = gzip.open('corpus_sent.txt.gz', 'wt')
f_out.writelines(f_in)
f_out.close()
f_in.close()

In [ ]:
#сохраняем предобработанные тексты в архив processed_corpus.txt.gz
!zcat < corpus_sent.txt.gz | python3 ./webvectors/preprocessing/rus_preprocessing_udpipe.py | gzip > processed_corpus_sent.txt.gz

UDPipe model not found. Downloading...

Loading the model...
Processing input...


In [ ]:
import gzip
f=gzip.open('processed_corpus_sent.txt.gz','rt')
sents_tagged=f.readlines()
f.close()
print(len(sents_tagged), len(all_sents))

26004 26004


In [ ]:
import wget, zipfile, gensim
model_url = 'http://vectors.nlpl.eu/repository/20/220.zip'
m = wget.download(model_url)
with zipfile.ZipFile("220.zip", 'r') as zip_ref:
    zip_ref.extractall("word2vec")

In [ ]:
import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format("word2vec/model.bin", binary=True)

In [ ]:
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, LSTM, Bidirectional
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Input, Embedding, Activation, Flatten, Dense, concatenate
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
df_sents_tagged = pd.concat([pd.DataFrame(sents_tagged[:len(sents_tagged) // 2]),
                 pd.DataFrame(sents_tagged[len(sents_tagged) // 2:])], axis=1)

In [ ]:
df_sents_tagged['score'] = df.score
df_sents_tagged.columns = ['sents_l', 'sents_r', 'score']

np.random.seed(42)
msk = np.random.rand(len(df_sents_tagged)) < 0.8
df_train_tagged = df_sents_tagged[msk]
df_test_tagged = df_sents_tagged[~msk]

In [ ]:
train_texts, test_texts = df_train_tagged[['sents_l', 'sents_r']], df_test_tagged[['sents_l', 'sents_r']]
train_labels, test_labels = df_train_tagged.score, df_test_tagged.score

In [ ]:
print("tokenizing input data...")
tokenizer = Tokenizer(lower=False, char_level=False, filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokenizer.fit_on_texts(train_texts.values.flatten())

tokenizing input data...


In [ ]:
#преобразуем тексты в последовательность номеров токенов
word_seq_train = tokenizer.texts_to_sequences(pd.concat([df_train_tagged['sents_l'], df_train_tagged['sents_r']], axis=0).values.flatten())
print("dictionary size: ", len(tokenizer.word_index))
print(train_texts.iloc[0])
print(word_seq_train[1])

dictionary size:  22759
sents_l    поскольку_SCONJ такой_DET правило_NOUN устанав...
sents_r    вы_PRON защищать_VERB свой_DET собственный_ADJ...
Name: 0, dtype: object
[52, 1, 24, 12, 117, 73, 263, 4393, 2137, 2000, 38, 7, 107, 131, 2909, 4, 4394]


In [ ]:
max_seq_len = 0
for t in word_seq_train:
  if len(t)>max_seq_len:
    max_seq_len = len(t)

print("Максимальная длина текста: ", max_seq_len)

Максимальная длина текста:  160


In [ ]:
tokenizer.index_word

{1: 'в_ADP',
 2: 'и_CCONJ',
 3: 'на_ADP',
 4: 'что_SCONJ',
 5: 'не_PART',
 6: 'с_ADP',
 7: 'быть_AUX',
 8: 'который_PRON',
 9: 'он_PRON',
 10: 'они_PRON',
 11: 'как_SCONJ',
 12: 'год_NOUN',
 13: 'то_PRON',
 14: 'это_PRON',
 15: 'по_ADP',
 16: 'этот_DET',
 17: 'мочь_VERB',
 18: 'для_ADP',
 19: 'к_ADP',
 20: 'из_ADP',
 21: 'свой_DET',
 22: 'за_ADP',
 23: 'xx_NUM',
 24: 'xxxx_NUM',
 25: 'я_PRON',
 26: 'о_ADP',
 27: 'мы_PRON',
 28: 'но_CCONJ',
 29: 'или_CCONJ',
 30: 'от_ADP',
 31: 'а_CCONJ',
 32: 'человек_NOUN',
 33: 'такой_DET',
 34: 'вы_PRON',
 35: 'время_NOUN',
 36: 'весь_DET',
 37: 'так_ADV',
 38: 'она_PRON',
 39: 'тот_DET',
 40: 'компания_NOUN',
 41: 'и_PART',
 42: 'же_PART',
 43: 'другой_ADJ',
 44: 'являться_VERB',
 45: 'что_PRON',
 46: 'более_ADV',
 47: 'также_ADV',
 48: 'если_SCONJ',
 49: 'один_NUM',
 50: 'становиться_VERB',
 51: 'до_ADP',
 52: 'когда_SCONJ',
 53: 'страна_NOUN',
 54: 'работа_NOUN',
 55: 'чтобы_SCONJ',
 56: 'x_NUM',
 57: 'самый_ADJ',
 58: '_SYM',
 59: 'xxx_NUM',
 60

In [ ]:
#делаем последовательности равной длины
# !pip install keras_preprocessing
# from keras_preprocessing import sequence
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_train[:10]

array([[    0,     0,     0, ...,   392,     6,  1104],
       [    0,     0,     0, ...,  2909,     4,  4394],
       [    0,     0,     0, ..., 13284,     1, 13285],
       ...,
       [    0,     0,     0, ...,   913,    67,   379],
       [    0,     0,     0, ...,   351,  2204,  2138],
       [    0,     0,     0, ...,   495,     3,    94]], dtype=int32)

In [ ]:
#то же самое для тестовой выборки
word_seq_test = tokenizer.texts_to_sequences(pd.concat([df_test_tagged['sents_l'], df_test_tagged['sents_r']], axis=0).values.flatten())
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)
word_seq_test[:10]

array([[   0,    0,    0, ..., 2393,  272,  228],
       [   0,    0,    0, ...,  356,   12, 4584],
       [   0,    0,    0, ...,    3, 8909, 1573],
       ...,
       [   0,    0,    0, ...,  958,   19,  166],
       [   0,    0,    0, ...,  563, 2678,  937],
       [   0,    0,    0, ...,   79,  202,  206]], dtype=int32)

In [ ]:
#делаем матрицу векторных представлений слов
import numpy as np
print('preparing embedding matrix...')
words_not_found = []
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, w2v_model.vector_size))
for word, i in tokenizer.word_index.items():
  if w2v_model.has_index_for(word):
    embedding_vector = w2v_model.get_vector(word)
    embedding_matrix[i] = embedding_vector
  else:
    words_not_found.append(word)
print('number of null word embeddings: %d' % len(words_not_found))

preparing embedding matrix...
number of null word embeddings: 5977


In [ ]:
from keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
shared_model = Sequential()
optimizer = Adam(lr=5e-6)
keras.utils.set_random_seed(42)
shared_model.add(Embedding(len(tokenizer.word_index)+1, 300,
              weights=[embedding_matrix], input_length=max_seq_len, trainable=False))

shared_model.add(LSTM(128, return_sequences=True))
shared_model.add(Dropout(0.2))
shared_model.add(LSTM(64, return_sequences=True))
shared_model.add(Dropout(0.2))
shared_model.add(LSTM(32))
# shared_model.add(Dropout(0.2))
# shared_model.add(LSTM(16))


left_input = Input(shape=(max_seq_len,))
right_input = Input(shape=(max_seq_len,))

x = shared_model(left_input)
# x = Dense(64, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(4, activation='relu')(x)
x = Model(inputs=left_input, outputs=x)

y = shared_model(right_input)
# y = Dense(64, activation='relu')(y)
y = Dense(16, activation='relu')(y)
y = Dense(8, activation='relu')(y)
y = Dense(4, activation='relu')(y)
y = Model(inputs=right_input, outputs=y)

combined = concatenate([x.output, y.output])

z = Dense(2, activation='relu')(combined)
z = Dense(1, activation='sigmoid')(z)



model = Model(inputs=[x.input, y.input], outputs=z)
model.compile(loss='mse', optimizer=optimizer,metrics=['mse', 'mae', r2_score], run_eagerly=True)

In [ ]:
# early_stopping = EarlyStopping(monitor='r2_score',min_delta=0.01, mode='min', patience=10, verbose=1)
# callbacks_list = [early_stopping]
batch_size = 128
n_epoch = 10

In [ ]:
hist = model.fit([word_seq_train[:len(word_seq_train) // 2],
                  word_seq_train[len(word_seq_train) // 2:]],
                  train_labels.values, batch_size=batch_size,
                  epochs=n_epoch, validation_data=([word_seq_test[:len(word_seq_test) // 2],
                                                    word_seq_test[len(word_seq_test) // 2:]], test_labels.values),
                  shuffle=True, validation_batch_size=batch_size)

Epoch 1/10


82/82 [==============================] - 32s 251ms/step - loss: 0.2477 - mse: 0.2477 - mae: 0.4964 - r2_score: 0.0014 - val_loss: 0.2470 - val_mse: 0.2470 - val_mae: 0.4764 - val_r2_score: -0.0016
Epoch 2/10
82/82 [==============================] - 22s 264ms/step - loss: 0.2365 - mse: 0.2365 - mae: 0.4757 - r2_score: 0.0458 - val_loss: 0.2368 - val_mse: 0.2368 - val_mae: 0.4714 - val_r2_score: 0.0420
Epoch 3/10
82/82 [==============================] - 20s 245ms/step - loss: 0.2269 - mse: 0.2269 - mae: 0.4596 - r2_score: 0.0853 - val_loss: 0.2350 - val_mse: 0.2350 - val_mae: 0.4626 - val_r2_score: 0.0516
Epoch 4/10
82/82 [==============================] - 20s 247ms/step - loss: 0.2174 - mse: 0.2174 - mae: 0.4424 - r2_score: 0.1233 - val_loss: 0.2344 - val_mse: 0.2344 - val_mae: 0.4576 - val_r2_score: 0.0516
Epoch 5/10
82/82 [==============================] - 24s 291ms/step - loss: 0.2037 - mse: 0.2037 - mae: 0.4221 - r2_score: 0.1791 - val_loss: 0.2386 - val_mse: 0.2386 - val_mae: 0.449

KeyboardInterrupt: 

In [ ]:
# model.predict([word_seq_test[:len(word_seq_test) // 2], word_seq_test[len(word_seq_test) // 2:]])

81/81 [==============================] - 7s 85ms/step


array([[0.711821  ],
       [0.6994402 ],
       [0.80346113],
       ...,
       [0.85642844],
       [0.37021825],
       [0.8891131 ]], dtype=float32)

In [ ]:
from sklearn.linear_model._bayes import BayesianRidge
# from sklearn.model_selection import GridSearchCV

In [ ]:

# br = BayesianRidge()

# param_grid = {
#     'n_iter': [100, 200, 300],
#     'alpha_1': [1e-6, 1e-5, 1e-4],
#     'alpha_2': [1e-6, 1e-5, 1e-4],
#     'lambda_1': [1e-6, 1e-5, 1e-4],
#     'lambda_2': [1e-6, 1e-5, 1e-4]
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(br, param_grid, cv=5)
# grid_search.fit(X_train, y_train)

In [ ]:
# grid_search

In [ ]:
# br.fit(X_train, y_train)

BayesianRidge(n_iter=1000, tol=1e-05)

In [ ]:
# br.score(X_test, y_test)

0.13000861933993169

In [ ]:
import fasttext.util
fasttext.util.download_model('ru', if_exists='ignore')
ft = fasttext.load_model('cc.ru.300.bin')

In [ ]:

from simpletransformers.language_representation import RepresentationModel
sents_l = df.sents_l.to_list()
sents_r = df.sents_r.to_list()
model_args = ClassificationArgs(num_train_epochs=1)
model_args.save_best_model = True
model_args.learning_rate = 3e-5
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs =  1
model_args.regression = True
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "r2"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluation_during_training = True
model_args.evaluate_during_training_steps = 200
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 42
model_args.overwrite_output_dir = True
model = RepresentationModel("bert",
                            "deeppavlov/rubert-base-cased",
                            args=model_args,
                            use_cuda=True,
                            ignore_mismatched_sizes=True,
    )
sents_l_vectors = model.encode_sentences(sents_l, combine_strategy="mean")
sents_r_vectors = model.encode_sentences(sents_r, combine_strategy="mean")
# assert sents_l_vectors.shape == (2, 768) # one sentence embedding per sentence

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("deeppavlov/rubert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at deeppavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
train_sents = []
for i in df_train.sents_l:
    train_sents.append(i)
for j in df_train.sents_r:
    train_sents.append(j)

test_sents = []
for i in df_eval.sents_l:
    test_sents.append(i)
for j in df_eval.sents_r:
    test_sents.append(j)

In [ ]:
train_embs = model.encode(train_sents)

In [ ]:
test_embs = model.encode(test_sents)

In [ ]:
X_train = pd.concat([pd.DataFrame(train_embs[:len(train_embs) // 2]),
                  pd.DataFrame(train_embs[
                    len(train_embs) // 2:])], axis=1)
y_train = df_train.score

In [ ]:
X_test = pd.concat([pd.DataFrame(test_embs[:len(test_embs) // 2]),
                  pd.DataFrame(test_embs[len(test_embs) // 2:])], axis=1)
y_test = df_eval.score

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
# from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.utils import all_estimators

def get_all_regressors_sklearn():

    estimators = all_estimators(type_filter='regressor')

    all_regs = []
    for name, RegClass in estimators:
        print('Appending', name)
        try:
            if name != 'QuantileRegressor':
                reg = RegClass()
                all_regs.append(reg)
        except Exception as e:
            pass
    return all_regs

all_regs = get_all_regressors_sklearn()
print(all_regs)

Appending ARDRegression
Appending AdaBoostRegressor
Appending BaggingRegressor
Appending BayesianRidge
Appending CCA
Appending DecisionTreeRegressor
Appending DummyRegressor
Appending ElasticNet
Appending ElasticNetCV
Appending ExtraTreeRegressor
Appending ExtraTreesRegressor
Appending GammaRegressor
Appending GaussianProcessRegressor
Appending GradientBoostingRegressor
Appending HistGradientBoostingRegressor
Appending HuberRegressor
Appending IsotonicRegression
Appending KNeighborsRegressor
Appending KernelRidge
Appending Lars
Appending LarsCV
Appending Lasso
Appending LassoCV
Appending LassoLars
Appending LassoLarsCV
Appending LassoLarsIC
Appending LinearRegression
Appending LinearSVR
Appending MLPRegressor
Appending MultiOutputRegressor
Appending MultiTaskElasticNet
Appending MultiTaskElasticNetCV
Appending MultiTaskLasso
Appending MultiTaskLassoCV
Appending NuSVR
Appending OrthogonalMatchingPursuit
Appending OrthogonalMatchingPursuitCV
Appending PLSCanonical
Appending PLSRegression

In [ ]:
X_train.columns = list(range(2 * 768))
X_test.columns = list(range(2 * 768))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
br = BayesianRidge()
lr = LinearRegression()
rfr = RandomForestRegressor(max_depth=2, random_state=0)

In [ ]:
rfr.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, random_state=0)

In [ ]:
br.fit(X_train, y_train)
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
from sklearn.linear_model import LassoLarsCV

In [ ]:
lasso = LassoLarsCV()

In [ ]:
lasso.fit(X_train, y_train)
lasso.score(X_test, y_test)

0.12301883402242719

In [ ]:
print(mean_absolute_error(y_test, lasso.predict(X_test)))
print(mean_squared_error(y_test, lasso.predict(X_test)))

0.43529198886245685
0.2192444686944087


In [ ]:
lr.score(X_test, y_test)

0.04405758617511757

In [ ]:
br.score(X_test, y_test)
br

0.13000876238629078

In [ ]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,0.328589,0.066195,-0.072625,0.135831,0.147369,0.026050,0.133850,0.024346,0.051270,0.305405,...,0.313086,-0.091115,-0.372719,-0.626881,0.105949,-0.184461,-0.359245,0.121846,0.602402,0.066203
1,0.101856,-0.528432,0.120811,-0.140357,0.305210,-0.173015,-0.271754,0.528842,-0.080881,-0.277301,...,0.036743,0.173897,-0.246651,-0.572409,-0.012130,0.330934,0.181845,-0.120944,0.589087,-0.335009
2,0.311145,-0.051073,0.016035,-0.209217,-0.243683,-0.127069,-0.175936,0.019341,-0.067313,0.077838,...,0.034481,0.119228,-0.068886,0.029083,-0.157886,-0.132243,0.001046,0.260112,0.163426,-0.186429
3,-0.067615,-0.508994,0.855734,0.065686,0.381888,0.102892,0.047972,-0.128109,0.083804,-0.677496,...,0.064602,-0.019688,-0.466747,-0.566742,0.362483,0.183491,-0.281036,0.511567,0.533687,-0.317412
4,0.077588,-0.155616,0.387244,-0.039153,0.137589,-0.186893,0.141328,0.200919,-0.331721,-0.452377,...,0.236981,0.128142,-0.539688,-0.841900,0.375824,0.036728,-0.203872,0.564283,0.008191,-0.636188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,0.324997,-0.010453,0.060624,-0.019704,0.006747,0.340397,0.091645,-0.033532,0.114492,-0.304755,...,0.041441,-0.169280,-0.553341,-0.739145,0.600230,0.188554,-0.070500,0.115897,0.097320,-0.211041
2577,0.011920,-0.392341,0.267803,-0.118613,0.154177,0.201716,-0.001567,0.175031,-0.186953,-0.158041,...,0.301989,0.232797,-0.443267,0.189927,0.505713,0.263640,0.027917,-0.030577,0.218354,-0.131008
2578,0.690894,-0.300938,0.422822,0.228840,-0.019143,-0.014181,-0.286053,0.110875,0.224963,0.231803,...,0.199000,-0.122176,-0.234737,-0.833904,0.338658,0.009143,-0.138530,0.383687,0.374221,0.039947
2579,0.117535,-0.233679,0.263740,-0.129544,0.074055,-0.253156,-0.047248,0.335486,-0.353927,0.365470,...,0.181399,0.271532,-0.238916,-0.300293,0.534691,0.283385,-0.258308,0.385268,0.672916,-0.224663


In [ ]:
print(mean_absolute_error(y_test, br.predict(X_test)))
print(mean_squared_error(y_test, br.predict(X_test)))

0.43499303300616377
0.21749699316152316


In [ ]:
rfr.score(X_test, y_test)

0.043530344488455164

In [ ]:
for reg in all_regs:
    try:
        reg.fit(X_train, y_train)
        print(type(reg), reg.score(X_test, y_test))
    except Exception:
        print(type(reg), 'no')

<class 'sklearn.linear_model._bayes.ARDRegression'> 0.10764135600106883
<class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'> 0.01578452733200908
<class 'sklearn.ensemble._bagging.BaggingRegressor'> 0.0021504589932411777
<class 'sklearn.linear_model._bayes.BayesianRidge'> 0.1300087509527863
<class 'sklearn.cross_decomposition._pls.CCA'> no
<class 'sklearn.tree._classes.DecisionTreeRegressor'> -0.8272056174431845
<class 'sklearn.dummy.DummyRegressor'> -5.842075443407424e-06
<class 'sklearn.linear_model._coordinate_descent.ElasticNet'> -5.8421575621636634e-06
<class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'> 0.123179257996061
<class 'sklearn.tree._classes.ExtraTreeRegressor'> -0.8272056174431845
<class 'sklearn.ensemble._forest.ExtraTreesRegressor'> 0.08683345792922226
<class 'sklearn.linear_model._glm.glm.GammaRegressor'> no
<class 'sklearn.gaussian_process._gpr.GaussianProcessRegressor'> -0.9961330239746709
<class 'sklearn.ensemble._gb.GradientBoostingRegressor'> 

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


<class 'sklearn.linear_model._huber.HuberRegressor'> 0.0014764450845295984
<class 'sklearn.isotonic.IsotonicRegression'> no
<class 'sklearn.neighbors._regression.KNeighborsRegressor'> -0.12155399595528737


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=2.95659e-08): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)


<class 'sklearn.kernel_ridge.KernelRidge'> 0.05004469293008307
<class 'sklearn.linear_model._least_angle.Lars'> 0.017028593542563364
<class 'sklearn.linear_model._least_angle.LarsCV'> 0.09076337951188274
<class 'sklearn.linear_model._coordinate_descent.Lasso'> -5.8421575621636634e-06
<class 'sklearn.linear_model._coordinate_descent.LassoCV'> 0.12297405926831395
<class 'sklearn.linear_model._least_angle.LassoLars'> -5.8421575621636634e-06
<class 'sklearn.linear_model._least_angle.LassoLarsCV'> 0.1230188402418324
<class 'sklearn.linear_model._least_angle.LassoLarsIC'> 0.1230573569283051
<class 'sklearn.linear_model._base.LinearRegression'> 0.04397984562490276


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


<class 'sklearn.svm._classes.LinearSVR'> -0.19120534999146055
<class 'sklearn.neural_network._multilayer_perceptron.MLPRegressor'> -0.32076318952224314
<class 'sklearn.linear_model._coordinate_descent.MultiTaskElasticNet'> no
<class 'sklearn.linear_model._coordinate_descent.MultiTaskElasticNetCV'> no
<class 'sklearn.linear_model._coordinate_descent.MultiTaskLasso'> no
<class 'sklearn.linear_model._coordinate_descent.MultiTaskLassoCV'> no
<class 'sklearn.svm._classes.NuSVR'> 0.09544517749360626
<class 'sklearn.linear_model._omp.OrthogonalMatchingPursuit'> 0.09941979188142702
<class 'sklearn.linear_model._omp.OrthogonalMatchingPursuitCV'> 0.09818307412721006
<class 'sklearn.cross_decomposition._pls.PLSCanonical'> no
<class 'sklearn.cross_decomposition._pls.PLSRegression'> 0.10855890099789889
<class 'sklearn.linear_model._passive_aggressive.PassiveAggressiveRegressor'> -0.6492822867215149
<class 'sklearn.linear_model._glm.glm.PoissonRegressor'> 0.0536618626371117


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


<class 'sklearn.linear_model._ransac.RANSACRegressor'> 0.009540741733283342
<class 'sklearn.neighbors._regression.RadiusNeighborsRegressor'> -0.9961330239752517


KeyboardInterrupt: 

In [ ]:
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
lf = LinearRegression().fit(X=pd.concat([pd.DataFrame(sents_l_vectors)[msk], pd.DataFrame(sents_r_vectors)[msk]], axis=1)
,y=df[msk].score)



In [ ]:
y_pred = lf.predict(pd.concat([pd.DataFrame(sents_l_vectors)[~msk], pd.DataFrame(sents_r_vectors)[~msk]], axis=1))

In [ ]:
lf.score(pd.concat([pd.DataFrame(sents_l_vectors)[~msk], pd.DataFrame(sents_r_vectors)[~msk]], axis=1),df[~msk].score )

0.03193349308273019

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
mean_absolute_error(df[~msk].score, y_pred)

0.4294172462071619

In [ ]:
mean_squared_error(df[~msk].score, y_pred)

0.24201571847137784

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# rfr = RandomForestRegressor(random_state=42).fit(X=pd.concat([pd.DataFrame(sents_l_vectors)[msk], pd.DataFrame(sents_r_vectors)[msk]], axis=1)
# ,y=df[msk].score)

In [ ]:
from sklearn import tree

In [ ]:
clf = tree.DecisionTreeRegressor(random_state=42)

In [ ]:
clf = clf.fit(X=pd.concat([pd.DataFrame(sents_l_vectors)[msk], pd.DataFrame(sents_r_vectors)[msk]], axis=1) ,y=df[msk].score)

In [ ]:
y_pred_ = clf.predict(X=pd.concat([pd.DataFrame(sents_l_vectors)[~msk], pd.DataFrame(sents_r_vectors)[~msk]], axis=1) )

In [ ]:
clf.score(pd.concat([pd.DataFrame(sents_l_vectors)[~msk], pd.DataFrame(sents_r_vectors)[~msk]], axis=1),df[~msk].score )

-0.8582014718527382

In [ ]:
mean_absolute_error(df[~msk].score, y_pred_)

0.46454862456412244

In [ ]:
mean_squared_error(df[~msk].score, y_pred_)

0.46454862456412244

In [ ]:
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# regr.fit(pd.concat([pd.DataFrame(sents_l_vectors), pd.DataFrame(sents_r_vectors)], axis=1)
# , df.score)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [ ]:
regr.predict([])

In [ ]:

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.DataFrame(df_train)
train_df.columns = ["text_a", "text_b", "labels"]

eval_df = pd.DataFrame(df_eval)
eval_df.columns = ["text_a", "text_b", "labels"]
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

model_args.save_best_model = False
model_args.learning_rate = 3e-5
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs =  1
model_args.regression = True
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "r2"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluation_during_training = True
model_args.evaluate_during_training_steps = 150
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 42
# model_args.wandb_project = "Simple Sweep"
# model_args.use_multiprocessing = True
#
model_args.overwrite_output_dir = True
model = ClassificationModel("bert",
                            "ai-forever/sbert_large_nlu_ru",
                            num_labels=1,
                            args=model_args,
                            use_cuda=True,
                            ignore_mismatched_sizes=True,

                            )
#
# # Train the model
model.train_model(train_df=train_df,
                  eval_df=eval_df,
                  mae=mean_absolute_error,
                  mse=mean_squared_error,
                  rmse=lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False),
                  r2=r2_score,
                  )
# # Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df=eval_df,
    mae=mean_absolute_error,
    mse=mean_squared_error,
    rmse=lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False,
                                                   ),
    r2=r2_score,
    output_dir='eval_rubert_1ep_default',
)
#
# # Make predictions with the model
# predictions, raw_outputs = model.predict(
#     [
#         [
#             "Legolas was an expert archer",
#             "Legolas was taller than Gimli",
#         ]
#     ]
# )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/323 [00:00<?, ?it/s]

In [ ]:

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.DataFrame(df_train)
train_df.columns = ["text_a", "text_b", "labels"]

eval_df = pd.DataFrame(df_eval)
eval_df.columns = ["text_a", "text_b", "labels"]
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

model_args.save_best_model = False
model_args.learning_rate = 5e-6
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs =  1
model_args.regression = True
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "r2"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluation_during_training = True
model_args.evaluate_during_training_steps = 150
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 42
# model_args.wandb_project = "Simple Sweep"
# model_args.use_multiprocessing = True
#
model_args.overwrite_output_dir = True
model = ClassificationModel("roberta",
                            "ai-forever/ruRoberta-large",
                            num_labels=1,
                            args=model_args,
                            use_cuda=True,
                            ignore_mismatched_sizes=True,
                            )
#
# # Train the model
model.train_model(train_df=train_df,
                  eval_df=eval_df,
                  mae=mean_absolute_error,
                  mse=mean_squared_error,
                  rmse=lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False),
                  r2=r2_score,
                  )
# # Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(
#     eval_df=eval_df,
#     mae=mean_absolute_error,
#     mse=mean_squared_error,
#     rmse=lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False,
#                                                    ),
#     r2=r2_score,
#     output_dir='eval_rubert_1ep_default',
# )
#
# # Make predictions with the model
# predictions, raw_outputs = model.predict(
#     [
#         [
#             "Legolas was an expert archer",
#             "Legolas was taller than Gimli",
#         ]
#     ]
# )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

(1303,
 defaultdict(list,
             {'global_step': [150, 300, 450, 600, 750, 900, 1050, 1200, 1303],
              'train_loss': [0.18340757489204407,
               0.38796788454055786,
               0.46504437923431396,
               0.201406791806221,
               0.20719853043556213,
               0.2833641469478607,
               0.11993452906608582,
               0.26718416810035706,
               0.18299676477909088],
              'mae': [0.49293856175543493,
               0.4930485899436386,
               0.47527579277903936,
               0.4554891841257325,
               0.4414530980961122,
               0.4398604370004121,
               0.4320859303080911,
               0.4282629263581162,
               0.4257758974520932],
              'mse': [0.3293749090814505,
               0.3032205587643401,
               0.34072643029705657,
               0.23568022828000673,
               0.22904435669740641,
               0.2244176595501383,
              

In [ ]:
model = ClassificationModel('roberta', 'outputs/best_model',
                            use_cuda=True)

In [ ]:
from simpletransformers.language_representation import RepresentationModel
sents_l = df.sents_l.to_list()
sents_r = df.sents_r.to_list()
model_args = ClassificationArgs(num_train_epochs=1)
model_args.save_best_model = True
model_args.learning_rate = 3e-5
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs =  1
model_args.regression = True
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "r2"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluation_during_training = True
model_args.evaluate_during_training_steps = 100
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 42
model_args.overwrite_output_dir = True
model = RepresentationModel("roberta",
                            "deeppavlov/rubert-base-cased",
                            args=model_args,
                            use_cuda=True,
                            ignore_mismatched_sizes=True,
    )
sents_l_vectors = model.encode_sentences(sents_l, combine_strategy="mean")
sents_r_vectors = model.encode_sentences(sents_r, combine_strategy="mean")
# assert sents_l_vectors.shape == (2, 768) # one sentence embedding per sentence

AttributeError: 'ClassificationModel' object has no attribute 'encode_sentences'

In [ ]:

# input_ids, attention_masks  = [], []
# for i in df[['sentence1', 'sentence2']].values:
#   d = tokenizer(i[0], i[1], padding=True,truncation=True)
#   input_ids.append(d['input_ids'])
#   attention_masks.append(d['attention_mask'])



In [ ]:
text = open('text_not_in_model.txt').read()

In [ ]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

model = ClassificationModel('bert', 'outputs/best_model',
                            use_cuda=True)

In [ ]:
predictions, raw_outputs = model.predict(
    [['Тиверзин шел по путям в направлении к городу.',
      'Навстречу ему попадались люди, шедшие с получкою из конторы.']])

# predictions, raw_outputs = model.predict(
#     [['Тиверзин шел по путям в направлении к городу.',
#       'Они простились и пошли в разные стороны.']])

# predictions, raw_outputs = model.predict([["По итогам сезона главный тренер Мануэль Пеллегрини был уволен с поста за неудовлетворительные результаты.",
  # "После непродолжительных переговоров главным тренером Реала назначен португалец Жозе Моуринью"]])
# predictions, raw_outputs = model.predict([["Это не просто мультик для детей."
# ,"Нечто большее."]])
# predictions, raw_outputs = model.predict(
    # [["Чем глубже погружаешься во что-то, тем сильнее ужасаешься, как много ты не знаешь.",
      # "По сравнению с теми, кто изучил что-то не глубоко и теперь думает, что он в теме  уши."]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_outputs


array(0.94433594)

In [ ]:
sent_pairs = []
sents = sent_tokenize(text)
for i in range(len(sents)-1):
  sent_pairs.append([sents[i], sents[i+1]])

In [ ]:
preds = []
for p in sent_pairs:

  predictions, raw_outputs = model.predict(
    [p])
  preds.append(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
np.mean(preds)

0.6913227754480699

In [ ]:
raw_outputs

array(0.93798828)

In [ ]:

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")

transformers_logger.setLevel(logging.WARNING)

train_df = pd.DataFrame(df_train)
train_df.columns = ["text_a", "text_b", "labels"]

eval_df = pd.DataFrame(df_eval)
eval_df.columns = ["text_a", "text_b", "labels"]
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

model_args.save_best_model = True
model_args.learning_rate = 3e-5
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs =  1
model_args.regression = True
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "r2"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluation_during_training = True
model_args.evaluate_during_training_steps = 100
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 42
# model_args.wandb_project = "Simple Sweep"
# model_args.use_multiprocessing = True
#
model_args.overwrite_output_dir = True
model = ClassificationModel("roberta",
                            "ai-forever/ruRoberta-large",
                            num_labels=1,
                            args=model_args,
                            use_cuda=True,
                            ignore_mismatched_sizes=True,

                            )
#
# # Train the model
model.train_model(train_df=train_df,
                  eval_df=eval_df,
                  mae=mean_absolute_error,
                  mse=mean_squared_error,
                  rmse=lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False),
                  r2=r2_score,
                  output_dir='train_rubert_1ep_64tbs_64ebs'
                  )
# # Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df=eval_df,
    mae=mean_absolute_error,
    mse=mean_squared_error,
    rmse=lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False,
                                                   ),
    r2=r2_score,
    output_dir='eval_rubert_1ep_default',
)
#
# # Make predictions with the model
# predictions, raw_outputs = model.predict(
#     [
#         [
#             "Legolas was an expert archer",
#             "Legolas was taller than Gimli",
#         ]
#     ]
# )


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/323 [00:00<?, ?it/s]